In [ ]:
import logging
from dotenv import load_dotenv

load_dotenv()

# Get the root logger
logger = logging.getLogger()
# Create a handler to output log messages to the console
handler = logging.StreamHandler()
# Create a formatter to format log messages
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [2]:
from local_llms import Llama4Maverick
from lmflux.flow import (
    new_toolbox,
    create_agent,
    tool
)
from lmflux.agents.sessions import Session
from lmflux.agents.structure import Agent
from lmflux.core.components import (SystemPrompt, Message, Conversation)

# Creating a new tool
@tool
def add(a: int, b: int) -> int:
    """Adds two numbes together"""
    return {"result": a + b}

def callback_tool_use(agent: Agent, tool_call, result, session: Session):
    print("Agent used some tool", tool_call, result, session)
    session.context.set("raw_add_result", result)

def conversation_update_callback(agent: Agent, conversation: Conversation):
    print(agent.agent_id, 'Got its conversation updated.', len(conversation), 'message on its context window')

def adder_act(agent:Agent, session:Session):
    a = session.context.get("a")
    b = session.context.get("b")
    message = Message(
        "user", f"What is {a}+{b}?"
    )
    response = agent.conversate(message, session)
    session.context.set("result", response.content)

# Set up the agent
system_prompt = SystemPrompt('general.user_preference')
adder_llm = Llama4Maverick(system_prompt)

toolbox = new_toolbox()
toolbox.add_to_toolbox(add)

adder_agent = (
    create_agent(adder_llm, "adder")
        .with_tools(toolbox)
        .with_custom_act(adder_act)
        .with_tool_callback(callback_tool_use)
        .with_conversation_update_callback(conversation_update_callback)
        .build()
)

# Call the agent
session = Session()
session.context.set("a", 985)
session.context.set("b", 1265)

adder_agent.act(session)

session.context.get("result"), session.context.get("raw_add_result")

BadRequestError: [{'type': 'int_parsing', 'loc': ('body', 'max_tokens'), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'None'}]